In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Import mechanical data ###

In [2]:
data = pd.read_csv('s0068sa03cal5_data_rp', skiprows = [1], sep = '\t') 

### Plot mechanical data vs sync ###

In [4]:
%matplotlib qt


### SETUP FIGURE ###
fig, ax = plt.subplots(figsize = (17,8))
fig.subplots_adjust(right=0.7)

ax.set_title('exp. s0068sa03cal5', fontsize = 16)

twin1 = ax.twinx()
twin2 = ax.twinx()

twin2.spines.right.set_position(("axes", 1.1))


### PLOT Normal stress, H-displacement and trigger sync (Arduino pulses) ###

p1, = ax.plot(data.records_na[4400:444000],data.normal_stress_MPa[4400:444000], c = 'cornflowerblue')
p2, = twin1.plot(data.records_na[4400:444000],-data.lt_ec_mm1[4400:444000], c = 'firebrick')
p3, = twin2.plot(data.records_na[4400:444000],data.sync[4400:444000], c = 'grey')

ax.set_xlabel("Time [s]", fontsize = 14)
ax.set_ylabel("Normal stress [MPa]", fontsize = 14)
twin1.set_ylabel('H-displacement [mm]', fontsize = 14)
twin2.set_ylabel('Trigger sync [V]', fontsize = 14)

ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p2.get_color())
twin2.yaxis.label.set_color(p3.get_color())

tkw = dict(size=4, width=1.5)
ax.tick_params(axis='y', colors=p1.get_color(), **tkw)
twin1.tick_params(axis='y', colors=p2.get_color(), **tkw)
twin2.tick_params(axis='y', colors=p3.get_color(), **tkw)
ax.tick_params(axis='x', **tkw)

#plt.savefig('ns_sync_test.pdf', dpi = 300)
plt.show();



In [2]:
data_w = pd.read_csv('C:\\Users\\miche\\Documents\\Università\\PhD\\test_waveforms\\fw\\data_tsv_file\\27_25_kn.bscan.tsv', encoding = 'iso8859', sep = '\t', skiprows = 4, header = None).dropna()


In [8]:
%matplotlib qt

winlen = 1000
noise_cut = 200
wf_number = 5
fs = 50
time_length = 50
time = np.arange(0, time_length, 1/fs)

filtered_data_w = np.zeros((len(data_w), len(time)-noise_cut))

for wf_number in range(0,len(data_w)):
    data_w_cut = data_w.iloc[wf_number][noise_cut:]
    time_length = time[noise_cut:]

    amp_spectrum = np.abs(np.fft.fft(data_w_cut))
    phase_spectrum = np.angle(np.fft.fft(data_w_cut))


    signal_freqs = np.fft.fftfreq(len(amp_spectrum), 1 / fs)
    pos_freq = signal_freqs > 0

    spectrum_han = np.hanning(winlen)
    spectrum_han_start = spectrum_han[round(winlen/2):]
    spectrum_han_end = spectrum_han[:round(winlen/2)]


    filter = np.zeros(len(amp_spectrum))
    filter[0:len(spectrum_han_start)] = spectrum_han_start
    filter[len(amp_spectrum)-len(spectrum_han_end):] = spectrum_han_end

    filtered_amp_spectrum = filter * amp_spectrum

    filtered_fourier = filtered_amp_spectrum*np.exp(1j*phase_spectrum)
    filtered_signal = (np.fft.ifft(filtered_fourier))

    filtered_data_w[wf_number, :] = filtered_signal



#plt.plot(time_length, data_w_cut)


#plt.plot(time_length, filtered_signal)

#plt.plot(filter)
#plt.plot(spectrum_han_start)
#plt.plot(spectrum_han_end)



#plt.plot(signal_freqs, spectrum, marker = 'o')

C:\Users\miche\AppData\Local\Temp\ipykernel_9056\1831881189.py:37: ComplexWarning: Casting complex values to real discards the imaginary part
  filtered_data_w[wf_number, :] = filtered_signal
c:\Users\miche\anaconda3\envs\friction\Lib\site-packages\matplotlib\cbook\__init__.py:1335: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [4]:
conc = np.concatenate((np.arange(0,50), np.arange(15000,15050)))

In [9]:
fig,ax1 = plt.subplots(ncols=2,figsize=(15,8))
ax1 = plt.subplot()
cmap = plt.get_cmap('seismic')
im = ax1.imshow(filtered_data_w[conc], aspect='auto',origin='upper',interpolation='none',cmap = cmap, extent=[0, 50,100,0])

cbar= fig.colorbar(im,pad=0.1)
cbar.set_label("Relative Amplitude", fontsize = 14)

ax1.set_xlabel('Wf_time [$\mu s$]', fontsize = 12)
ax1.set_ylabel('Wf_number [$\mu s$]', fontsize = 12)



fig.tight_layout()

C:\Users\miche\AppData\Local\Temp\ipykernel_9056\248298553.py:2: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax1 = plt.subplot()
